In [ ]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.41.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1
!pip install -qqq evaluate==0.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 100.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 113.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:

import json
import os
from pprint import pprint
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [ ]:
MODEL_NAME = "facebook/dpr-ctx_encoder-single-nq-base"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = DPRContextEncoder.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = DPRContextEncoderTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00012-of-00015.safetensors:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

model-00013-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/828M [00:00<?, ?B/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 10>:10                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:485 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   482 │   │   │   │   class_ref, pretrained_model_name_or_path, **hub_kwargs, **kwargs           │
│   483 │   │   │   )                                                                              │
│   484 │   │   │   _ = hub_kwargs.pop("code_revision", None)                                      │
│ ❱ 485 │   │   │   return model_class.from_pretrained(                                            │
│   486 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   487 │   │   │   )                                                                              │
│   488 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2815 in from_pretrained   │
│                                                                                                  │
│   2812 │   │   │   │   │   key: device_map[key] for key in device_map.keys() if key not in modu  │
│   2813 │   │   │   │   }                                                                         │
│   2814 │   │   │   │   if "cpu" in device_map_without_lm_head.values() or "disk" in device_map_  │
│ ❱ 2815 │   │   │   │   │   raise ValueError(                                                     │
│   2816 │   │   │   │   │   │   """                                                               │
│   2817 │   │   │   │   │   │   Some modules are dispatched on the CPU or the disk. Make sure yo  │
│   2818 │   │   │   │   │   │   the quantized model. If you want to dispatch the model on the CP  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to 
fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a 
custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-
cpu-and-gpu
                        for more details.

In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model

In [ ]:
prompt = """
<human>: 25-year-old primigravida presents to her physician for a routine prenatal visit  she is at 34 weeks gestation, as confirmed by an ultrasound examination  she has no complaints.
what is the best treatment for this patient?
<assistant>:
""".strip()

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: 25-year-old primigravida presents to her physician for a routine prenatal visit  she is at 34 weeks gestation, as confirmed by an ultrasound examination  she has no complaints.
what is the best treatment for this patient?
<assistant>: The best treatment for this patient would depend on the specific circumstances and medical history. It is recommended to consult with a healthcare provider for personalized recommendations.
CPU times: user 5.11 s, sys: 334 ms, total: 5.45 s
Wall time: 8.13 s


# Prep dataset

In [ ]:
data = load_dataset("csv", data_files="processed_data_medium.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e48de48f991f598a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'instruction', 'input', 'output', 'question'],
        num_rows: 2996
    })
})

In [ ]:
data["train"][0]

{'Unnamed: 0': 0,
 'instruction': 'please give a diagnosis based on the given information',
 'input': 'a 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination  she states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract  she otherwise feels well and is followed by a doctor for her pregnancy  her temperature is 97 7°f (36 5°c), blood pressure is 122/77 mmhg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air',
 'output': 'nitrofurantoin',
 'question': ' what is the best treatment for this patient?'}

In [ ]:
def generate_prompt(data_point):
  if data_point["input"] is None or data_point["question"] is None or data_point["output"] is None:
    print(data_point["input"])
    print(data_point["question"])
    print(data_point["output"])
    return f"""""".strip()

  return f"""
<human>: {data_point["input"]+ ". " + data_point["question"]}
<assistant>: {data_point["output"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/2996 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['Unnamed: 0', 'instruction', 'input', 'output', 'question', 'input_ids', 'attention_mask'],
    num_rows: 2996
})

# Finetune the model

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

Step,Training Loss
1,1.855000
2,1.753200
3,1.906700
4,1.462100
5,1.683000
6,1.776700
7,1.769100
8,1.662100
9,1.729300
10,1.774300


TrainOutput(global_step=749, training_loss=1.6734814421038762, metrics={'train_runtime': 5219.4014, 'train_samples_per_second': 0.574, 'train_steps_per_second': 0.144, 'total_flos': 9632469449740800.0, 'train_loss': 1.6734814421038762, 'epoch': 1.0})

# Save trained model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "azgeb/air-ws23-falcon-7b"

In [ ]:
model.push_to_hub(
   PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:288: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:288: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/azgeb/air-ws23-falcon-7b/commit/ae7c2923dd8b22dd008aa904cf466583834e5d9e', commit_message='Upload model', commit_description='', oid='ae7c2923dd8b22dd008aa904cf466583834e5d9e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:5                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'bnb_config' is not defined

# Run the finetuned model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time

device = "cuda:0"

prompt = """
<human>: 25-year-old primigravida presents to her physician for a routine prenatal visit  she is at 34 weeks gestation, as confirmed by an ultrasound examination  she has no complaints.
what is the best treatment for this patient?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: 25-year-old primigravida presents to her physician for a routine prenatal visit  she is at 34 weeks gestation, as confirmed by an ultrasound examination  she has no complaints.
what is the best treatment for this patient?
<assistant>: magnesium sulfate 1g, administered intravenously, 1 hour prior to delivery.
" |  " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | " | "
CPU times: user 18.6 s, sys: 50.5 ms, total: 18.7 s
Wall time: 18.8 s


# Evaluation

In [ ]:
import evaluate

## Metrics

#### Exact Match

https://huggingface.co/spaces/evaluate-metric/exact_match

In [ ]:
exact_match_metric = evaluate.load("exact_match")

#### SQuAD v2

https://huggingface.co/spaces/evaluate-metric/squad_v2

In [ ]:
squad_metric = evaluate.load("squad_v2")